<a href="https://colab.research.google.com/github/ajdachi/ajdachi/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-exp' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhlbGxvIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiSGVsbG8hIEknbSBoZXJlIHRvIGFzc2lzdCB5b3Ugd2l0aCB5b3VyIGZpbmFuY2lhbCBhbmQgcmVhbCBlc3RhdGUgaW52ZXN0bWVudCBpbnF1aXJpZXMuIEhvdyBjYW4gSSBoZWxwIHlvdSB0b2RheT9cXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6InN1cmUgeW91IGNhbiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IkdyZWF0ISBUbyBzdGFydCwgY291bGQgeW91IHRlbGwgbWUgYSBsaXR0bGUgYWJvdXQgeW91ciByaXNrIHRvbGVyYW5jZT8gRm9yIGV4YW1wbGUsIGFyZSB5b3UgY29tZm9ydGFibGUgd2l0aCBwb3RlbnRpYWxseSBoaWdoZXIgcmlzayBmb3IgaGlnaGVyIHJldHVybnMsIG9yIGRvIHlvdSBwcmVmZXIgbW9yZSBzdGFibGUsIGxvd2VyLXJpc2sgaW52ZXN0bWVudHM/XFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJJIGNhbnQgZmlndXJlIG91dCBteSByaXNrIHRvcmVsYW5jZSB0byBzYXkgdGhlIHRydWUsIGJ1dCBJIHdvdWxkIGxpa2UgdG8gZ2V0IGhpZ2hlciByZXR1cm4sIHdoYXQgaXMgeW91ciBhZHZpY2U/In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiT2theSwgSSB1bmRlcnN0YW5kLiBJdCdzIG5vdCB1bmNvbW1vbiB0byBmaW5kIGl0IGNoYWxsZW5naW5nIHRvIHBpbnBvaW50IHlvdXIgZXhhY3QgcmlzayB0b2xlcmFuY2UsIGFuZCBpdCdzIGFsc28gbmF0dXJhbCB0byBkZXNpcmUgaGlnaGVyIHJldHVybnMuXFxuXFxuR2l2ZW4geW91ciBwcmVmZXJlbmNlIGZvciBoaWdoZXIgcmV0dXJucywgd2UgY2FuIGV4cGxvcmUgaW52ZXN0bWVudCBvcHRpb25zIHRoYXQgdHlwaWNhbGx5IGNhcnJ5IGEgYml0IG1vcmUgcmlzayBidXQgYWxzbyBoYXZlIHRoZSBwb3RlbnRpYWwgZm9yIGdyZWF0ZXIgZ3Jvd3RoLiBUaGlzIG1pZ2h0IGluY2x1ZGUgYSBwb3J0Zm9saW8gd2l0aCBhIGhpZ2hlciBhbGxvY2F0aW9uIHRvIHN0b2NrcyBvciByZWFsIGVzdGF0ZSwgd2hpY2ggaGF2ZSBoaXN0b3JpY2FsbHkgb2ZmZXJlZCBoaWdoZXIgcmV0dXJucyBjb21wYXJlZCB0byBtb3JlIGNvbnNlcnZhdGl2ZSBvcHRpb25zIGxpa2UgYm9uZHMuXFxuXFxuSG93ZXZlciwgc2luY2UgeW91J3JlIHVuc3VyZSBvZiB5b3VyIHJpc2sgdG9sZXJhbmNlLCBpdCdzIGVzc2VudGlhbCB0byBwcm9jZWVkIGNhcmVmdWxseS4gSGVyZeKAmXMgd2hhdCBJIHN1Z2dlc3Q6XFxuXFxuKioxLiBVbmRlcnN0YW5kaW5nIHRoZSBUcmFkZS1vZmY6KiogQmVmb3JlIHdlIGRpdmUgaW4sIGl0J3MgaW1wb3J0YW50IHRvIHVuZGVyc3RhbmQgdGhlIHJpc2stcmV3YXJkIHJlbGF0aW9uc2hpcC4gSGlnaGVyIHBvdGVudGlhbCByZXR1cm5zIG9mdGVuIGNvbWUgd2l0aCBoaWdoZXIgcG90ZW50aWFsIGxvc3Nlcy4gSSB3YW50IHRvIG1ha2Ugc3VyZSB5b3UncmUgY29tZm9ydGFibGUgd2l0aCB0aGUgcG9zc2liaWxpdHkgb2YgZmx1Y3R1YXRpb25zIGluIHlvdXIgaW52ZXN0bWVudCB2YWx1ZS5cXG5cXG4qKjIuIEEgQmFsYW5jZWQgQXBwcm9hY2g6KiogU2luY2UgeW91IHdhbnQgaGlnaGVyIHJldHVybnMgYnV0IHlvdSBhcmUgdW5zdXJlIGFib3V0IHlvdXIgcmlzayB0b2xlcmFuY2UsIEkgd291bGQgcmVjb21tZW5kIGEgYmFsYW5jZWQgcG9ydGZvbGlvLiBJdCBjb3VsZCBpbmNsdWRlIGEgbWl4IG9mOlxcbiAgICAqICAgKipTdG9ja3M6KiogT2ZmZXIgaGlnaCBncm93dGggcG90ZW50aWFsIGJ1dCBhcmUgbW9yZSB2b2xhdGlsZS5cXG4gICAgKiAgICoqQm9uZHM6KiogUHJvdmlkZSBtb3JlIHN0YWJpbGl0eSBhbmQgbG93ZXIgcmV0dXJucywgYWN0aW5nIGFzIGEgYnVmZmVyLlxcbiAgICAqICAgKipNdXR1YWwgRnVuZHMgb3IgRVRGczoqKiBEaXZlcnNpZmllZCBpbnZlc3RtZW50cyB0aGF0IHNwcmVhZCByaXNrLlxcbiAgICAqICAgKipSZWFsIEVzdGF0ZToqKiBDYW4gb2ZmZXIgYm90aCBpbmNvbWUgYW5kIGFwcHJlY2lhdGlvbi5cXG5cXG4qKjMuIEdyYWR1YWwgRXhwb3N1cmU6KiogWW91IGRvbid0IGhhdmUgdG8ganVtcCBpbiBoZWFkZmlyc3QuIFN0YXJ0IHdpdGggYSBtb2RlcmF0ZWx5IGFnZ3Jlc3NpdmUgYWxsb2NhdGlvbiBhbmQgZ3JhZHVhbGx5IGFkanVzdCBhcyB5b3UgYmVjb21lIG1vcmUgY29tZm9ydGFibGUgYW5kIGdhaW4gZXhwZXJpZW5jZS5cXG5cXG4qKjQuIENvbnNpZGVyIFlvdXIgR29hbHM6KiogVG8gcHJvdmlkZSB0aGUgYmVzdCBhZHZpY2UsIEkgd291bGQgc3RpbGwgbGlrZSB0byB1bmRlcnN0YW5kIHlvdXIgZmluYW5jaWFsIGdvYWxzLCBpdCBjYW4gcHJvdmlkZSBhIGJldHRlciBwZXJzcGVjdGl2ZSwgZG8geW91IHdhbnQgdG8gYWNoaWV2ZSByZXRpcmVtZW50LCB3ZWFsdGggYWNjdW11bGF0aW9uLCBpbmNvbWUgZ2VuZXJhdGlvbiBvciBvdGhlcj9cXG5cXG4qKkRpc2NsYWltZXI6KiogUGxlYXNlIHJlbWVtYmVyIHRoYXQgSSBhbSBhbiBBSSBhbmQgY2FuIG9ubHkgcHJvdmlkZSBnZW5lcmFsIGFkdmljZS4gSXQncyBpbXBvcnRhbnQgdG8gY29uc3VsdCB3aXRoIGEgcXVhbGlmaWVkIGZpbmFuY2lhbCBhZHZpc29yIGZvciBwZXJzb25hbGl6ZWQgcmVjb21tZW5kYXRpb25zLlxcblxcbldvdWxkIHlvdSBsaWtlIG1lIHRvIGVsYWJvcmF0ZSBvbiBhbnkgb2YgdGhlc2UgcG9pbnRzIG9yIGV4cGxvcmUgYW55IHBhcnRpY3VsYXIgaW52ZXN0bWVudCB0eXBlIGZ1cnRoZXI/IEFsc28sIGlmIHlvdSBjb3VsZCBwcm92aWRlIHNvbWUgaW5mb3JtYXRpb24gYWJvdXQgeW91ciBmaW5hbmNpYWwgZ29hbHMsIHRoYXQgd291bGQgYmUgaGVscGZ1bCB0byBnaXZlIHlvdSBhIG1vcmUgdGFpbG9yZWQgYWR2aWNlLlxcbiJ9XX0seyJyb2xlIjoidXNlciIsInBhcnRzIjpbeyJ0ZXh0IjoiSSBoYXZlICQ1MDAwMDAgdGhhdCBJIHdvdWxkIGxpa2UgdG8gaW52ZXN0OyBhbmQgSSBsaXZlIGluIFRhbnphbmlhLiB3aGF0IGlzIHlvdXIgYWR2aWNlPyJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IlRoYXQncyBncmVhdCB0aGF0IHlvdSBoYXZlICQ1MDAsMDAwIHRvIGludmVzdCwgYW5kIGl0J3MgaW1wb3J0YW50IHRvIGNvbnNpZGVyIHlvdXIgbG9jYXRpb24gaW4gVGFuemFuaWEgd2hlbiBtYWtpbmcgZmluYW5jaWFsIGRlY2lzaW9ucy4gSGVyZeKAmXMgbXkgYWR2aWNlLCBrZWVwaW5nIGluIG1pbmQgeW91ciBkZXNpcmUgZm9yIGhpZ2hlciByZXR1cm5zIHdoaWxlIGFsc28gYmVpbmcgbWluZGZ1bCBvZiB5b3VyIGxvY2F0aW9uOlxcblxcbioqMS4gVW5kZXJzdGFuZGluZyB0aGUgTG9jYWwgSW52ZXN0bWVudCBMYW5kc2NhcGUgaW4gVGFuemFuaWE6KipcXG4gICAgKiAgICoqRW1lcmdpbmcgTWFya2V0IENvbnNpZGVyYXRpb25zOioqIFRhbnphbmlhIGlzIGFuIGVtZXJnaW5nIG1hcmtldCwgd2hpY2ggcHJlc2VudHMgdW5pcXVlIG9wcG9ydHVuaXRpZXMgYW5kIGNoYWxsZW5nZXMuIFRoZXJlIGNvdWxkIGJlIGhpZ2hlciBncm93dGggcG90ZW50aWFsLCBidXQgYWxzbyBoaWdoZXIgdm9sYXRpbGl0eSBhbmQgcmlza3MgY29tcGFyZWQgdG8gbW9yZSBkZXZlbG9wZWQgbWFya2V0cy5cXG4gICAgKiAgICoqQ3VycmVuY3kgUmlzazoqKiBJbnZlc3RpbmcgaW4gZm9yZWlnbiBhc3NldHMgbWF5IGV4cG9zZSB5b3UgdG8gY3VycmVuY3kgZmx1Y3R1YXRpb25zIGJldHdlZW4gdGhlIFRhbnphbmlhbiBTaGlsbGluZyBhbmQgb3RoZXIgY3VycmVuY2llcy5cXG4gICAgKiAgICoqTG9jYWwgUmVndWxhdGlvbnM6KiogSXQncyB2aXRhbCB0byBiZSBhd2FyZSBvZiBsb2NhbCBpbnZlc3RtZW50IHJlZ3VsYXRpb25zIGFuZCB0YXhhdGlvbiBwb2xpY2llcy5cXG4gICAgKiAgICoqSW52ZXN0bWVudCBPcHRpb25zOioqIEZhbWlsaWFyaXplIHlvdXJzZWxmIHdpdGggaW52ZXN0bWVudCBvcHRpb25zIGF2YWlsYWJsZSBsb2NhbGx5IGFuZCBpbnRlcm5hdGlvbmFsbHksIGFuZCB0aGF0IGFyZSBhY2Nlc3NpYmxlIHRvIHlvdSBpbiBUYW56YW5pYS5cXG5cXG4qKjIuIFBvcnRmb2xpbyBEaXZlcnNpZmljYXRpb24gd2l0aCBhIEZvY3VzIG9uIEdyb3d0aDoqKlxcblxcbkdpdmVuIHlvdXIgZ29hbCBmb3IgaGlnaGVyIHJldHVybnMsIGEgZGl2ZXJzaWZpZWQgcG9ydGZvbGlvIGNvdWxkIGluY2x1ZGU6XFxuICAgICogICAqKkxvY2FsIEVxdWl0aWVzIChTdG9ja3MpOioqIEludmVzdGluZyBpbiBUYW56YW5pYW4gc3RvY2sgbWFya2V0IGNhbiBwcm92aWRlIGV4cG9zdXJlIHRvIGxvY2FsIGJ1c2luZXNzZXMuIEl0IGFsc28gYWxsb3dzIHlvdSB0byBjb250cmlidXRlIHRvIGxvY2FsIGdyb3d0aCBhbmQgaXQgYWxsb3dzIGZvciBkaXZpZGVuZCBwYXltZW50IGFuZCBjYXBpdGFsIGdhaW4uXFxuICAgICogICAqKkludGVybmF0aW9uYWwgRXF1aXRpZXMgKFN0b2Nrcyk6KiogSW5jbHVkaW5nIHN0b2NrcyBpbiBkZXZlbG9wZWQgbWFya2V0cyBjYW4gcHJvdmlkZSBkaXZlcnNpZmljYXRpb24gYW5kIGV4cG9zdXJlIHRvIGdsb2JhbCBncm93dGguIFRoaXMgY291bGQgYmUgZG9uZSB0aHJvdWdoIGEgYnJva2VyYWdlIGFjY291bnQgdGhhdCBnaXZlcyB5b3UgYWNjZXNzIHRvIHRoZSBpbnRlcm5hdGlvbmFsIG1hcmtldC5cXG4gICAgKiAgKipCb25kczoqKiBXaGlsZSB0aGV5IG1heSBvZmZlciBsb3dlciByZXR1cm5zLCB0aGV5IGNhbiBwcm92aWRlIHN0YWJpbGl0eSB0byB5b3VyIHBvcnRmb2xpby4gVGhlIGJvbmRzIGNhbiBiZSBpc3N1ZWQgYnkgdGhlIGdvdmVybm1lbnQgb3IgdGhlIHByaXZhdGUgc2VjdG9yLlxcbiAgICAqICAgKipSZWFsIEVzdGF0ZToqKiBJbnZlc3RpbmcgaW4gcmVhbCBlc3RhdGUgaW4gVGFuemFuaWEgY2FuIHByb3ZpZGUgcmVudGFsIGluY29tZSBhbmQgcHJvcGVydHkgYXBwcmVjaWF0aW9uLCBlc3BlY2lhbGx5IGluIGdyb3dpbmcgdXJiYW4gYXJlYXMuXFxuICAgICogICAqKk11dHVhbCBGdW5kcyBvciBFVEZzOioqIFRoZXNlIG9mZmVyIGRpdmVyc2lmaWNhdGlvbiB3aXRoaW4gZGlmZmVyZW50IGFzc2V0IGNsYXNzZXMsIGxvY2FsbHkgYW5kIGludGVybmF0aW9uYWxseS4gVGhlc2UgYXJlIG1hbmFnZWQgcHJvZmVzc2lvbmFsbHkuXFxuICAgICogICAqKkFsdGVybmF0aXZlIEludmVzdG1lbnRzOioqIERlcGVuZGluZyBvbiB5b3VyIGNvbWZvcnQgbGV2ZWwgYW5kIGxvY2FsIGF2YWlsYWJpbGl0eSwgb3B0aW9ucyBsaWtlIHByaXZhdGUgZXF1aXR5IG9yIGluZnJhc3RydWN0dXJlIHByb2plY3RzIGNhbiBhbHNvIGJlIGNvbnNpZGVyZWQuXFxuXFxuKiozLiBTcGVjaWZpYyBBZHZpY2UgYmFzZWQgb24geW91ciBMb2NhdGlvbioqXFxuXFxuKiAgICoqTG9jYWwgRXhwZXJ0aXNlOioqIENvbnNpZGVyIGNvbnN1bHRpbmcgYSBmaW5hbmNpYWwgYWR2aXNvciBpbiBUYW56YW5pYSB3aG8gdW5kZXJzdGFuZHMgdGhlIGxvY2FsIGludmVzdG1lbnQgbGFuZHNjYXBlIGFuZCByZWd1bGF0aW9ucy4gVGhleSBjYW4gcHJvdmlkZSBwZXJzb25hbGl6ZWQgYWR2aWNlIHRhaWxvcmVkIHRvIHlvdXIgY2lyY3Vtc3RhbmNlcy5cXG4qICAgKipSZWFsIEVzdGF0ZSBPcHBvcnR1bml0aWVzOioqIEV4cGxvcmUgcmVhbCBlc3RhdGUgaW4gYXJlYXMgd2l0aCBoaWdoIGdyb3d0aCBwb3RlbnRpYWwgaW4gVGFuemFuaWEsIHN1Y2ggYXMgRGFyIGVzIFNhbGFhbSwgQXJ1c2hhLCBvciBvdGhlciBkZXZlbG9waW5nIGNpdGllcy4gRm9jdXMgb24gYXJlYXMgd2hlcmUgdGhlcmUgaXMgaW5mcmFzdHJ1Y3R1cmUgZGV2ZWxvcG1lbnQuXFxuKiAgICoqTG9jYWwgQnVzaW5lc3NlczoqKiBZb3UgY2FuIGV4cGxvcmUgdGhlIG9wdGlvbiBvZiBpbnZlc3RpbmcgaW4gbG9jYWwgY29tcGFuaWVzIG9yIHN1cHBvcnRpbmcgbmV3IHByb2plY3RzLlxcbiogICAqKkdvdmVybm1lbnQgQm9uZHM6KiogSW52ZXN0aW5nIGluIFRhbnphbmlhbiBnb3Zlcm5tZW50IGJvbmRzIGNvdWxkIGJlIGEgc2FmZSB3YXkgdG8gZ2VuZXJhdGUgcmV0dXJucywgYnV0IHRoZXkgbWlnaHQgbm90IGJlIGFzIGhpZ2ggYXMgb3RoZXIgcmlza2llciBhc3NldHMuXFxuKiAgICoqSW5mbGF0aW9uIEhlZGdlOioqIEluIFRhbnphbmlhLCB3aGVyZSBpbmZsYXRpb24gY2FuIGJlIGEgY29uY2VybiwgaXQgaXMgZXNzZW50aWFsIHRvIGludmVzdCBpbiBhc3NldHMgdGhhdCBjYW4gaGVkZ2UgaW5mbGF0aW9uIHN1Y2ggYXMgcmVhbCBlc3RhdGUgYW5kIGdvbGQuXFxuKiAgICoqQ3VycmVuY3kgQ29uc2lkZXJhdGlvbjoqKiBDb25zaWRlciB0aGF0IGludmVzdGluZyBpbiBmb3JlaWduIGN1cnJlbmNpZXMgbWF5IGV4cG9zZSB5b3UgdG8gZXhjaGFuZ2UgcmF0ZXMgZmx1Y3R1YXRpb25zLlxcblxcbioqNC4gUmlzayBNYW5hZ2VtZW50OioqXFxuICAgICogICAqKkRpdmVyc2lmaWNhdGlvbjoqKiBBcyBtZW50aW9uZWQsIGRpdmVyc2lmeSB5b3VyIHBvcnRmb2xpbyBhY3Jvc3MgZGlmZmVyZW50IGFzc2V0IGNsYXNzZXMgYW5kIGdlb2dyYXBoaWVzIHRvIHJlZHVjZSByaXNrLlxcbiAgICAqICAgKipQaGFzZWQgSW52ZXN0bWVudDoqKiBSYXRoZXIgdGhhbiBpbnZlc3RpbmcgdGhlIGVudGlyZSBhbW91bnQgYXQgb25jZSwgeW91IGNhbiBjb25zaWRlciBpbnZlc3RpbmcgZ3JhZHVhbGx5IG92ZXIgdGltZSB0byB0YWtlIGFkdmFudGFnZSBvZiBtYXJrZXQgZmx1Y3R1YXRpb25zIChkb2xsYXIgY29zdCBhdmVyYWdpbmcpLlxcbiAgICAqICAgKipSZWd1bGFyIFJldmlldzoqKiBQZXJpb2RpY2FsbHkgcmV2aWV3IHlvdXIgcG9ydGZvbGlvIGFuZCBhZGp1c3QgeW91ciBzdHJhdGVneSBhcyBuZWNlc3NhcnksIGJhc2VkIG9uIG1hcmtldCBjb25kaXRpb25zIGFuZCB5b3VyIGZpbmFuY2lhbCBnb2Fscy5cXG5cXG4qKjUuIFRheCBDb25zaWRlcmF0aW9uczoqKlxcbiAgICAqICAgVW5kZXJzdGFuZCB0aGUgdGF4IGltcGxpY2F0aW9ucyBvZiB5b3VyIGludmVzdG1lbnRzIGluIFRhbnphbmlhLCBib3RoIGluIGxvY2FsIGFuZCBpbnRlcm5hdGlvbmFsLCBhcyB0aGlzIHdpbGwgaW1wYWN0IHRoZSBmaW5hbCByZXR1cm5zLlxcblxcbioqRGlzY2xhaW1lcjoqKiBQbGVhc2UgcmVtZW1iZXIgdGhhdCBJIGFtIGFuIEFJIGFuZCBjYW4gb25seSBwcm92aWRlIGdlbmVyYWwgYWR2aWNlLiBJdCdzIGltcG9ydGFudCB0byBjb25zdWx0IHdpdGggYSBxdWFsaWZpZWQgZmluYW5jaWFsIGFkdmlzb3Igd2hvIGhhcyBhIGRlZXAgdW5kZXJzdGFuZGluZyBvZiB0aGUgVGFuemFuaWFuIG1hcmtldCBmb3IgcGVyc29uYWxpemVkIHJlY29tbWVuZGF0aW9ucy5cXG5cXG5Xb3VsZCB5b3UgbGlrZSB0byBkaXNjdXNzIGFueSBvZiB0aGVzZSBwb2ludHMgZnVydGhlciwgb3Igd291bGQgeW91IGxpa2UgbWUgdG8gZXhwbG9yZSBhbnkgcGFydGljdWxhciBhc3NldCBjbGFzcyBpbiBtb3JlIGRldGFpbD9cXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IlRoYW5rIHlvdSB2ZXJ5IG11Y2ggaXQgd2FzIGhlbHBmdWwifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJZb3UncmUgdmVyeSB3ZWxjb21lISBJJ20gZ2xhZCBJIGNvdWxkIHByb3ZpZGUgc29tZSBoZWxwZnVsIGluc2lnaHRzLiBJZiB5b3UgaGF2ZSBhbnkgbW9yZSBxdWVzdGlvbnMgaW4gdGhlIGZ1dHVyZSBvciBuZWVkIGZ1cnRoZXIgYXNzaXN0YW5jZSB3aXRoIHlvdXIgZmluYW5jaWFsIGFuZCByZWFsIGVzdGF0ZSBpbnZlc3RtZW50IHBsYW5uaW5nLCBmZWVsIGZyZWUgdG8gcmVhY2ggb3V0LiBCZXN0IG9mIGx1Y2sgd2l0aCB5b3VyIGludmVzdG1lbnQgam91cm5leSFcXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")